In [1]:
import numpy as np
import pandas as pd
import scipy
import os
from sklearn.utils.extmath import stable_cumsum
import math
import matplotlib.pyplot as plt
import pickle
import functools
import scipy as sp
from sklearn.model_selection import StratifiedKFold, cross_validate
from scipy.spatial import distance
from sklearn.decomposition import PCA,qPCA
from tabulate import tabulate

from sklearn.QuantumUtility.Utility import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('kddcup.data_10_percent_corrected',header = None)
df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


In [3]:
df1 = df.drop(columns=[1,2,3,6,11,20,21])
df1.loc[df1[41] != 'normal.', 41] = 'attack'
df1

,0,4,5,7,8,9,10,12,13,14,...,32,33,34,35,36,37,38,39,40,41
0,0,181,5450,0,0,0,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,239,486,0,0,0,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,235,1337,0,0,0,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,219,1337,0,0,0,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,217,2032,0,0,0,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,310,1881,0,0,0,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,282,2286,0,0,0,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,203,1200,0,0,0,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,291,1200,0,0,0,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


In [4]:
def systematic_sampling(df,pop_length):
    df_normal=df[df[41]!='attack']
    step = int(len(df_normal)/pop_length)
    
    
    first_index = random.choice(np.arange(step))
    indexes = np.arange(first_index, len(df), step=step)[:5000]
    
    training_set = df_normal.iloc[indexes]
    
    mean_std = training_set.drop(columns=41).describe(include='all').loc[['mean','std']]
    
    index_res = list(set(df_normal.index)-set(training_set.index))
    test = df_normal.loc[index_res]
    t = df[df[41] =='attack'].sample(39674)
    test= test.append(t)

    return training_set,test,mean_std


In [5]:
train1,test1,mean1 = systematic_sampling(df1,5000)

train2,test2,mean2 = systematic_sampling(df1,5000)

train3,test3,mean3 = systematic_sampling(df1,5000)

train4,test4,mean4 = systematic_sampling(df1,5000)

train5,test5,mean5 = systematic_sampling(df1,5000)

In [6]:
mean_std = [mean1,mean2,mean3,mean4,mean5]
test = [test1,test2,test3,test4,test5]

# ---------------------------------------------------------------------------------------------------------------

# Trimming function

In [8]:
def trimming(n_trimming, df, gamma):

    for n_tr in range(n_trimming):
        mean_array = np.array(df.drop(columns=41).describe(include='all').loc[['mean','std']].iloc[0])
        cov = np.cov(df.drop(columns=41).T)
        Icov = np.linalg.pinv(cov)
        dist_list = []
        for i in range(len(df)):
            sample_i = np.array(df.drop(columns=41).iloc[i])
            dist =distance.mahalanobis(sample_i,mean_array,Icov)**2
            dist_list.append(dist)
            #print(i)
        threshold = int(gamma*len(df))
        dist_list=np.array(dist_list)
        x = np.argpartition(dist_list,-threshold)[-threshold:]
        names =[]
    
        for i in x:
            names.append(df.iloc[i].name)
        df = df.drop(index = names)
    return df


## Trimming 1

Trimming the whole dataset and then extract 5000 samples from the training set.

In [9]:

#trimming 1 time
df1_normal = df1[df1[41]!='attack']
train = trimming(1,df1_normal,0.005)
train_trim=train.sample(5000)
#train_trim,test_trim,mean_trim = systematic_sampling(df1,5000)
mean_std_trim = train_trim.drop(columns=41).describe(include='all').loc[['mean','std']]
##TEST SET

x = list(train_trim.index)
index_res = list(set(df1_normal.index)-set(x))
test_trim = df1_normal.loc[index_res]
t = df1[df1[41] =='attack'].sample(39674)
test_trim= test_trim.append(t)
test_trim


,0,4,5,7,8,9,10,12,13,14,...,32,33,34,35,36,37,38,39,40,41
1,0,239,486,0,0,0,0,0,0,0,...,19,1.00,0.00,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,235,1337,0,0,0,0,0,0,0,...,29,1.00,0.00,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,219,1337,0,0,0,0,0,0,0,...,39,1.00,0.00,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,217,2032,0,0,0,0,0,0,0,...,49,1.00,0.00,0.02,0.0,0.0,0.0,0.0,0.0,normal.
5,0,217,2032,0,0,0,0,0,0,0,...,59,1.00,0.00,0.02,0.0,0.0,0.0,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299144,0,1032,0,0,0,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,attack
95428,0,1032,0,0,0,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,attack
388301,0,0,0,0,0,0,0,0,0,0,...,15,0.06,0.06,0.00,0.0,1.0,1.0,0.0,0.0,attack
352935,0,0,0,0,0,0,0,0,0,0,...,2,0.01,0.07,0.00,0.0,1.0,1.0,0.0,0.0,attack


## Trimming 2

Trimming the 5000 connections of the training set

In [9]:
train1 = trimming(1,train1,0.005)
train2 = trimming(1,train2,0.005)
train3 = trimming(1,train3,0.005)
train4 = trimming(1,train4,0.005)
train5 = trimming(1,train5,0.005)

In [10]:
trains = [train1,train2,train3,train4,train5]

# ---------------------------------------------------------------------------------------------------------------

## Normalize training set and test set


In [11]:
scaler = StandardScaler()
for train_ in trains:
    for (columnName, columnData) in train_.drop(columns=41).iteritems():
        train_[columnName] = scaler.fit_transform(train_.drop(columns=41)[[columnName]])

In [12]:
#Remove mean and std from test set (Normalize test set)
tests =[]
for en, test_ in enumerate(test):
    test_ = (test_.drop(columns=41).sub(mean_std[en].loc['mean'],axis = 'columns')).div(mean_std[en].loc['std'],axis = 'columns').fillna(0)
    tests.append(test_)


# Saved Train/Test

In [4]:
trains=[]
tests=[]
test = []
for i in range(5):
    trains.append(pd.read_csv('trains'+str(i)).drop(columns='Unnamed: 0'))
    tests.append(pd.read_csv('tests'+str(i)).set_index('Unnamed: 0'))



In [5]:
for i in range(5):
    x = tests[i].index
    test.append(df1.loc[x])
test[0]

,0,4,5,7,8,9,10,12,13,14,...,32,33,34,35,36,37,38,39,40,41
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,0,181,5450,0,0,0,0,0,0,0,...,9,1.00,0.00,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,239,486,0,0,0,0,0,0,0,...,19,1.00,0.00,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,235,1337,0,0,0,0,0,0,0,...,29,1.00,0.00,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,219,1337,0,0,0,0,0,0,0,...,39,1.00,0.00,0.03,0.0,0.0,0.0,0.0,0.0,normal.
5,0,217,2032,0,0,0,0,0,0,0,...,59,1.00,0.00,0.02,0.0,0.0,0.0,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195319,0,1032,0,0,0,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,attack
446383,0,520,0,0,0,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,attack
356582,0,0,0,0,0,0,0,0,0,0,...,10,0.04,0.07,0.00,0.0,1.0,1.0,0.0,0.0,attack


# Fit PCA 

In [38]:
#Fit PCAs
pca30 = qPCA(svd_solver="full").fit(trains[0].drop(columns='41'),classic_ret_variance_components=0.30)

pca40 = qPCA(svd_solver="full").fit(trains[1].drop(columns='41'),classic_ret_variance_components=0.40)

pca50 = qPCA(svd_solver="full").fit(trains[2].drop(columns='41'),classic_ret_variance_components=0.50)

pca60 = qPCA(svd_solver="full").fit(trains[3].drop(columns='41'),classic_ret_variance_components=0.60)

pca70 = qPCA(svd_solver="full").fit(trains[4].drop(columns='41'),classic_ret_variance_components=0.70)

PCAs=[pca30,pca40,pca50,pca60,pca70]


In [13]:
pca30.explained_variance_

array([4.65200997e+00, 3.81257993e+00, 1.86116316e+00, 1.69345291e+00,
       1.59134659e+00, 1.26237817e+00, 1.07079483e+00, 1.03925052e+00,
       1.02120032e+00, 1.01683754e+00, 9.89818558e-01, 9.43599090e-01,
       9.19827674e-01, 8.80069697e-01, 8.12627698e-01, 5.76027458e-01,
       4.48860857e-01, 4.33295497e-01, 3.51716473e-01, 2.00870615e-01,
       1.05955635e-01, 1.02966615e-01, 8.92560885e-02, 8.45970472e-02,
       3.48741199e-02, 9.85012735e-03, 9.17447198e-31, 1.75878763e-31,
       7.13325531e-32, 2.67881362e-32, 2.67881362e-32, 2.67881362e-32,
       2.00097448e-32, 3.11469147e-33])

# Empirical Distribution

In [6]:
# Compute Quantile---> 1-\alpha1
alpha = [0.01,0.02,0.04,0.06,0.08,0.10]
quantils = []
for i in alpha:
    eq =  [1,-2,i]
    quantile = 1-np.round(np.roots(eq)[1],decimals=4)
    quantils.append(quantile)
quantils

[0.995, 0.9899, 0.9798, 0.9695, 0.9592, 0.9487]

Found outlier threshold using empirical distribution of \sum_q(y_i**2)/\lambda_i and of \sum_p-r+1(y_i**2)/\lambda_i

In [67]:
dictionary_major = {}
dictionary_minor  = {}
for e,pca in enumerate(PCAs):
    
    r = len(pca.explained_variance_[pca.explained_variance_ < 0.20])
    
    p = len(pca.explained_variance_)

    out_threshold_list_major = []
    out_threshold_list_minor = []
    emp_distribution_major = []
    emp_distribution_minor = []
    
    for j in range(len(trains[e].drop(columns='41'))):
        sample = np.array(trains[e].drop(columns='41').iloc[j])
        y = np.dot(sample,pca.components_.T)
        s_major = np.sum(y[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])
        
        s_minor = np.sum(y[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]]**2 / pca.explained_variance_[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]])

        emp_distribution_major.append(s_major)
        emp_distribution_minor.append(s_minor)

    for q in quantils:
        n_major = len(emp_distribution_major)
        n_minor = len(emp_distribution_minor)

        sort_major = sorted(emp_distribution_major)
        sort_minor = sorted(emp_distribution_minor)
        out_threshold_major = sort_major[int(n_major * q)]
        out_threshold_minor = sort_minor[int(n_minor * q)]
        out_threshold_list_major.append(out_threshold_major)
        out_threshold_list_minor.append(out_threshold_minor)

    dictionary_major.update({pca.components_retained_:out_threshold_list_major})
    dictionary_minor.update({pca.components_retained_:out_threshold_list_minor})


In [68]:
dictionary_major

{2: [22.197239064164407,
  19.99135862820585,
  18.197348601053637,
  17.001445027751092,
  16.0683386812283,
  15.010143914289939],
 4: [69.7749948782532,
  32.17587415121882,
  23.15469524671563,
  19.75679080825643,
  18.161205247493285,
  16.791437688340412],
 5: [118.91989297756282,
  57.71486878410796,
  48.49149985862438,
  29.534375138260447,
  22.505350523381708,
  19.550683048211365],
 7: [127.96508185755336,
  66.25735417557607,
  49.424201563129245,
  41.121314855752345,
  31.486294005681483,
  26.12570163807499],
 10: [240.59948776903184,
  139.89956814669893,
  70.55634718070701,
  56.878906533400226,
  44.54120936228148,
  29.069106302796193]}

In [69]:
#Compute Confusion Matrix using outlier threshold
# Experiment =0 -> use only major components. 
#Experiment = 1 -> use both major and minor components

experiment = 1

if experiment == 0:
    classification_res = {}
    for e,pca in enumerate(PCAs):
    
        acc_list  = []
        for threshold_major in dictionary_major[pca.components_retained_]:
            print(threshold_major)
            TP =[]
            FP = []
            TN = []
            FN = []

            for j in range(len(tests[e])):
                sample = np.array(tests[e].iloc[j])
                y = np.dot(sample, pca.components_.T)

                sum_major = np.sum(y[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])

                if sum_major > threshold_major:
                    if test[e].iloc[j][41] == 'attack':
                        TP.append(j)
                    else:
                        FP.append(j)
                else:
                    if sum_major <= threshold_major:

                        if test[e].iloc[j][41] == 'attack':
                            FN.append(j)
                        else:
                            TN.append(j)

            accuracy = (len(TP))/(len(TP)+len(FN))
            print(accuracy)
            acc_list.append(accuracy)

        classification_res.update({pca.components_retained_:acc_list})

else:
    print('exp1')
    classification_res = {}
    for e,pca in enumerate(PCAs):
        r = len(pca.explained_variance_[pca.explained_variance_ < 0.20])
        p = len(pca.explained_variance_)

        acc_list  = []
        for threshold_major,threshold_minor in zip(dictionary_major[pca.components_retained_],dictionary_minor[pca.components_retained_]):
            print(threshold_major,threshold_minor)
            TP =[]
            FP = []
            TN = []
            FN = []

            for j in range(len(tests[e])):
                sample = np.array(tests[e].iloc[j])

                y = np.dot(sample, pca.components_.T)

                sum_major = np.sum(y[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])

                sum_minor = np.sum(y[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]]**2 / pca.explained_variance_[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]])



                if sum_major > threshold_major or sum_minor > threshold_minor:
                    if test[e].iloc[j][41] == 'attack':
                        TP.append(j)
                    else:
                        FP.append(j)
                else:
                    if sum_major <= threshold_major and sum_minor <= threshold_minor:

                        if test[e].iloc[j][41] == 'attack':
                            FN.append(j)
                        else:
                            TN.append(j)

            accuracy = (len(TP))/(len(TP)+len(FN))
            print(accuracy)
            acc_list.append(accuracy)

        classification_res.update({pca.components_retained_:acc_list})



exp1
22.197239064164407 224.59356683131793
0.9427584816252458
19.99135862820585 148.32209883024575
0.9610828250239452
18.197348601053637 64.72320606526893
0.9891868730150728
17.001445027751092 47.4282310649588
0.9901950899833645
16.0683386812283 38.26045873662737
0.9907244038917175
15.010143914289939 25.30241924774484
0.991556182890558
69.7749948782532 195.35870475606765
0.9857589353228815
32.17587415121882 113.5902663876304
0.9870444119574533
23.15469524671563 54.72942998923343
0.9880022180773302
19.75679080825643 41.14578133339469
0.9894389272571458
18.161205247493285 32.62736756567716
0.9903715279528155
16.791437688340412 22.221135217266692
0.9917578262842164
118.91989297756282 200.79463382068343
0.9853556485355649
57.71486878410796 123.95904202010576
0.9865403034733075
48.49149985862438 59.51454784399132
0.9873216716237334
29.534375138260447 49.0237503609418
0.9877753692594646
22.505350523381708 28.13865016219134
0.9897918031960478
19.550683048211365 20.221232928322227
0.9909764581

In [36]:
# First experiment
components = [i for i in classification_res]
if experiment == 0:

    headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70']
    one_perc = [classification_res[i][0] for i in components]
    two_perc = [classification_res[i][1] for i in components]
    four_perc = [classification_res[i][2] for i in components]
    six_perc = [classification_res[i][3] for i in components]
    eight_perc = [classification_res[i][4] for i in components]
    ten_perc = [classification_res[i][5] for i in components]


    one_perc.insert(0,'1%')
    two_perc.insert(0,'2%')
    four_perc.insert(0,'4%')
    six_perc.insert(0,'6%')
    eight_perc.insert(0,'8%')
    ten_perc.insert(0,'10%')

    print(tabulate([one_perc, two_perc, four_perc, six_perc, eight_perc, ten_perc], headers=headers))
else:
    #Second experiment
    headers = ['FAR','PCC']
    one_perc = [classification_res[i][0] for i in components]
    two_perc = [classification_res[i][1] for i in components]
    four_perc = [classification_res[i][2] for i in components]
    six_perc = [classification_res[i][3] for i in components]
    eight_perc = [classification_res[i][4] for i in components]
    ten_perc = [classification_res[i][5] for i in components]

    one_perc_avg = np.mean(one_perc)
    one_perc_std = np.std(one_perc)

    two_perc_avg = np.mean(two_perc)
    two_perc_std = np.std(two_perc)

    four_perc_avg = np.mean(four_perc)
    four_perc_std = np.std(four_perc)

    six_perc_avg = np.mean(six_perc)
    six_perc_std = np.std(six_perc)


    eight_perc_avg = np.mean(eight_perc)
    eight_perc_std = np.std(eight_perc)

    ten_perc_avg = np.mean(ten_perc)
    ten_perc_std = np.std(ten_perc)

    one_str = [str(one_perc_avg)+' +/- '+str(one_perc_std)]
    two_str = [str(two_perc_avg)+' +/- '+str(two_perc_std)]
    four_str = [str(four_perc_avg)+' +/- '+str(four_perc_std)]
    six_str = [str(six_perc_avg)+' +/- '+str(six_perc_std)]
    eight_str =[ str(eight_perc_avg)+' +/- '+str(eight_perc_std)]
    ten_str = [str(ten_perc_avg)+' +/- '+str(ten_perc_std)]

    one_str.insert(0,'1%')
    two_str.insert(0,'2%')
    four_str.insert(0,'4%')
    six_str.insert(0,'6%')
    eight_str.insert(0,'8%')
    ten_str.insert(0,'10%')

    print(tabulate([one_str, two_str, four_str, six_str, eight_str, ten_str], headers=headers))


FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.671321  0.957632  0.932864  0.935224  0.931282
2%     0.688184  0.985287  0.972117  0.961742  0.931737
4%     0.711751  0.985692  0.984647  0.986083  0.960357
6%     0.713616  0.986779  0.985907  0.987298  0.985058
8%     0.713893  0.987259  0.987218  0.988006  0.986727
10%    0.714196  0.987436  0.987445  0.988993  0.994387


In [71]:
components = [i for i in classification_res]
headers = ['FAR','PCC']
one_perc = [classification_res[i][0] for i in components]
two_perc = [classification_res[i][1] for i in components]
four_perc = [classification_res[i][2] for i in components]
six_perc = [classification_res[i][3] for i in components]
eight_perc = [classification_res[i][4] for i in components]
ten_perc = [classification_res[i][5] for i in components]

one_perc_avg = np.mean(one_perc)
one_perc_std = np.std(one_perc)

two_perc_avg = np.mean(two_perc)
two_perc_std = np.std(two_perc)

four_perc_avg = np.mean(four_perc)
four_perc_std = np.std(four_perc)

six_perc_avg = np.mean(six_perc)
six_perc_std = np.std(six_perc)


eight_perc_avg = np.mean(eight_perc)
eight_perc_std = np.std(eight_perc)

ten_perc_avg = np.mean(ten_perc)
ten_perc_std = np.std(ten_perc)

one_str = [str(one_perc_avg)+' +/- '+str(one_perc_std)]
two_str = [str(two_perc_avg)+' +/- '+str(two_perc_std)]
four_str = [str(four_perc_avg)+' +/- '+str(four_perc_std)]
six_str = [str(six_perc_avg)+' +/- '+str(six_perc_std)]
eight_str =[ str(eight_perc_avg)+' +/- '+str(eight_perc_std)]
ten_str = [str(ten_perc_avg)+' +/- '+str(ten_perc_std)]

one_str.insert(0,'1%')
two_str.insert(0,'2%')
four_str.insert(0,'4%')
six_str.insert(0,'6%')
eight_str.insert(0,'8%')
ten_str.insert(0,'10%')

print(tabulate([one_str, two_str, four_str, six_str, eight_str, ten_str], headers=headers))


FAR    PCC
-----  --------------------------------------------
1%     0.9773046382009991 +/- 0.017277049377979803
2%     0.9817819997535467 +/- 0.01035542644800812
4%     0.9882759318536403 +/- 0.0006692154240590303
6%     0.9890678895858972 +/- 0.0008632678300970236
8%     0.9900158672421652 +/- 0.0004825865447686921
10%    0.9924270670645295 +/- 0.0017969092422338239


# Quantum

# ---------------------------------------------------------------------------------------------------------------
## No tomography

In [44]:
qpca30 = qPCA(svd_solver='full').fit(trains[0].drop(columns=41),p=0.30,estimate_all=True,delta=0.1,eps=0.2,theta=0.901703)
qpca40 = qPCA(svd_solver='full').fit(trains[1].drop(columns=41),p=0.40,estimate_all=True,delta=0.1,eps=0.2,theta =0.624750)
qpca50 = qPCA(svd_solver='full').fit(trains[2].drop(columns=41),p=0.50,estimate_all=True,delta=0.1,eps=0.2,theta = 0.575973)
qpca60 = qPCA(svd_solver='full').fit(trains[3].drop(columns=41),p=0.60,estimate_all=True,delta=0.1,eps=0.2,theta =0.475055)
qpca70 = qPCA(svd_solver='full').fit(trains[4].drop(columns=41),p=0.70,estimate_all=True,delta=0.1,eps=0.2,theta =0.474014)

QPCAs = [qpca30,qpca40,qpca50,qpca60,qpca70]


# ---------------------------------------------------------------------------------------------------------------
## Tomography

In [6]:
qpca30 = qPCA(svd_solver='full').fit(trains[0].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.30,estimate_all=True,delta=0.1,eps=0.2,tomography=True)
qpca40 = qPCA(svd_solver='full').fit(trains[1].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.40,estimate_all=True,delta=0.1,eps=0.2,tomography=True)
qpca50 = qPCA(svd_solver='full').fit(trains[2].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.50,estimate_all=True,delta=0.1,eps=0.2,tomography=True)
qpca60 = qPCA(svd_solver='full').fit(trains[3].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.60,estimate_all=True,delta=0.1,eps=0.2,tomography=True)
qpca70 = qPCA(svd_solver='full').fit(trains[4].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=0.70,estimate_all=True,delta=0.1,eps=0.2,tomography=True)



0.9038306630879078 [1.         0.90383066]
[-0.00120818]
0.6216121887167015 [1.         0.90401813 0.65768652 0.62161219]
[0.00065041]
0.5805121995824393 [1.         0.91742683 0.6496076  0.62880236 0.5805122 ]
[0.00467888]
0.478026933410716 [1.         0.89815519 0.67818982 0.62827273 0.58592385 0.50310174
 0.47802693]
[0.00028382]
0.4696950108308548 [1.         0.91742683 0.6496076  0.62880236 0.5805122  0.53282479
 0.48205212 0.4774751  0.47534135 0.46969501]
[0.00479146]


In [12]:
QPCAs = [qpca30,qpca40,qpca50,qpca60,qpca70]

In [11]:
qpca70.estimate_fs

[4.5379094024390945,
 3.788392691218734,
 2.002959869527456,
 1.8784308070135487,
 1.4852774090551457,
 1.243494607248263,
 1.1354180041891775,
 1.1116100942138385,
 1.0811474862622632]

In [48]:
qpca30.components_

array([[-7.97458110e-02, -2.37464063e-02, -1.26050085e-02, ...,
         1.49355293e-02,  4.37394995e-01,  4.35535375e-01],
       [ 2.96992605e-01,  3.82114063e-02, -2.70224202e-02, ...,
         2.10530848e-02,  8.82442527e-02,  8.45778157e-02],
       [-1.52799679e-03,  4.29442732e-02,  2.84256883e-02, ...,
         2.71756799e-01, -4.92857622e-02, -4.36327077e-02],
       ...,
       [ 0.00000000e+00, -2.42263945e-17, -8.86784755e-17, ...,
         7.46366479e-17, -1.01187266e-16,  9.61402993e-18],
       [-0.00000000e+00,  1.36978865e-16,  1.79278233e-16, ...,
         2.20249233e-16, -3.88297066e-16,  3.64398914e-16],
       [-0.00000000e+00,  2.74251618e-17,  7.16299611e-17, ...,
         8.35779691e-17,  1.39417231e-16,  1.53542170e-16]])

In [18]:
qpca30.estimate_right_sv

array([[-0.0591422 , -0.01687086, -0.01316407, -0.        , -0.        ,
        -0.00890985, -0.        , -0.        , -0.        , -0.        ,
        -0.01143222, -0.00405684, -0.        , -0.0075577 , -0.        ,
        -0.04439687, -0.04416732, -0.00295179, -0.00278297,  0.30830263,
         0.30661331,  0.05966372, -0.05631647,  0.03073891, -0.15440781,
         0.05600619,  0.09786502, -0.08622717, -0.00139148,  0.25207009,
        -0.00482025, -0.00368152,  0.30771172,  0.30600016],
       [ 0.20563162,  0.02162405, -0.06949071, -0.        , -0.        ,
         0.01492201, -0.        , -0.        , -0.        , -0.        ,
         0.04054454,  0.00880052, -0.        , -0.        , -0.        ,
        -0.11662354, -0.13024328, -0.02011646, -0.02130836,  0.06996278,
         0.06700834, -0.18760034,  0.17504803, -0.00139148,  0.01819603,
        -0.29235757, -0.29675679,  0.29335261,  0.26780572,  0.04271881,
         0.00511264,  0.00491964,  0.07080867,  0.06699389]])

In [15]:
dictionary_major = {}
dictionary_minor  = {}
for e,pca in enumerate(QPCAs):


    out_threshold_list_major = []
    out_threshold_list_minor = []
    emp_distribution_major = []
    
    
    for j in range(len(trains[e].drop(columns='41'))):
        sample = np.array(trains[e].drop(columns='41').iloc[j])
        y = np.dot(sample,pca.estimate_right_sv.T)
        s_major = np.sum(y**2 / pca.estimate_fs)

        emp_distribution_major.append(s_major)
        

    for q in quantils:
        n_major = len(emp_distribution_major)
        
        sort_major = sorted(emp_distribution_major)
        
        out_threshold_major = sort_major[int(n_major * q)]
        
        out_threshold_list_major.append(out_threshold_major)
        
    
    dictionary_major.update({pca:out_threshold_list_major})
    #dictionary_minor.update({pca.components_retained_:out_threshold_list_minor})


In [16]:
dictionary_major

{qPCA(svd_solver='full'): [10.554172602996656,
  9.490379475505451,
  8.643329270637466,
  8.055212842825537,
  7.622435255681397,
  7.116525111284617],
 qPCA(svd_solver='full'): [10.097980273250645,
  7.958136032846995,
  6.502951453863909,
  5.86914190617801,
  5.399259824988744,
  5.091122661211569],
 qPCA(svd_solver='full'): [30.49402227517458,
  10.672683975442734,
  6.292301923740932,
  5.182812785972688,
  4.619670663612691,
  4.242188043529786],
 qPCA(svd_solver='full'): [15.111955014814892,
  9.704986186384826,
  8.144498088952464,
  6.268768871926175,
  4.6102879038369675,
  3.6886191843631857],
 qPCA(svd_solver='full'): [22.025877314029117,
  13.72707192858178,
  6.893812641442821,
  5.746687222339921,
  4.0959637968664,
  3.0666442813933785]}

In [17]:

classification_res = {}
for e,pca in enumerate(QPCAs):
    
    acc_list  = []
    for threshold_major in dictionary_major[pca]:
        print(threshold_major)
        TP =[]
        FP = []
        TN = []
        FN = []

        for j in range(len(tests[e])):
            sample = np.array(tests[e].iloc[j])

            y = np.dot(sample,pca.estimate_right_sv.T)
            
            
            sum_major = np.sum(y**2 / pca.estimate_fs)
            
            if sum_major > threshold_major:
                if test[e].iloc[j][41] == 'attack':
                    TP.append(j)
                else:
                    FP.append(j)
            else:
                if sum_major <= threshold_major:

                    if test[e].iloc[j][41] == 'attack':
                        FN.append(j)
                    else:
                        TN.append(j)

        accuracy = (len(TP))/(len(TP)+len(FN))
        print(accuracy)
        acc_list.append(accuracy)

    classification_res.update({pca:acc_list})


10.554172602996656
0.6722712447220045
9.490379475505451
0.6907284265884555
8.643329270637466
0.7115875704786225
8.055212842825537
0.7127253419635408
7.622435255681397
0.7129781800713003
7.116525111284617
0.7132815858006119
10.097980273250645
0.9311591454936164
7.958136032846995
0.9371255214258627
6.502951453863909
0.9578055871571229
5.86914190617801
0.9701428390848186
5.399259824988744
0.985665529010239
5.091122661211569
0.9862469978510934
30.49402227517458
0.9323036477160697
10.672683975442734
0.9733309739882201
6.292301923740932
0.9857680932278369
5.182812785972688
0.9867792411334968
4.619670663612691
0.9871837002957607
4.242188043529786
0.9873100937839683
15.111955014814892
0.9542004048582996
9.704986186384826
0.9796558704453441
8.144498088952464
0.9862095141700404
6.268768871926175
0.9868674089068826
4.6102879038369675
0.9870445344129555
3.6886191843631857
0.9882591093117409
22.025877314029117
0.9313309061488673
13.72707192858178
0.9319377022653722
6.893812641442821
0.9773715614886

In [18]:
components = [i for i in classification_res]
#pcas = [i for i in classification_res]
#pcas

In [19]:
#Tomography delta = 0.1, eps = 0.2

headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70']
one_perc = [classification_res[i][0] for i in components]
two_perc = [classification_res[i][1] for i in components]
four_perc = [classification_res[i][2] for i in components]
six_perc = [classification_res[i][3] for i in components]
eight_perc = [classification_res[i][4] for i in components]
ten_perc = [classification_res[i][5] for i in components]


one_perc.insert(0,'1%')
two_perc.insert(0,'2%')
four_perc.insert(0,'4%')
six_perc.insert(0,'6%')
eight_perc.insert(0,'8%')
ten_perc.insert(0,'10%')

print(tabulate([one_perc, two_perc, four_perc, six_perc, eight_perc, ten_perc], headers=headers))

FAR       PCA30     PCA40     PCA50     PCA60     PCA70
-----  --------  --------  --------  --------  --------
1%     0.672271  0.931159  0.932304  0.9542    0.931331
2%     0.690728  0.937126  0.973331  0.979656  0.931938
4%     0.711588  0.957806  0.985768  0.98621   0.977372
6%     0.712725  0.970143  0.986779  0.986867  0.986094
8%     0.712978  0.985666  0.987184  0.987045  0.987004
10%    0.713282  0.986247  0.98731   0.988259  0.994463


# 2nd Experiment Quantum

In [7]:
qpca30 = qPCA(svd_solver='full').fit(trains[0].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.1,eps=0.2,tomography=True)
qpca40 = qPCA(svd_solver='full').fit(trains[1].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.1,eps=0.2,tomography=True)
qpca50 = qPCA(svd_solver='full').fit(trains[2].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.1,eps=0.2,tomography=True)
qpca60 = qPCA(svd_solver='full').fit(trains[3].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.1,eps=0.2,tomography=True)
qpca70 = qPCA(svd_solver='full').fit(trains[4].drop(columns='41'),theta_estimate=True,eps_theta=0.005,p=1,estimate_all=True,delta=0.1,eps=0.2,tomography=True)

1.8828331708221996e-17 [1.00000000e+00 9.03830663e-01 6.63063972e-01 6.24413960e-01
 6.04266248e-01 5.76869685e-01 5.01479000e-01 4.72646662e-01
 4.70028342e-01 4.64541182e-01 4.60064245e-01 4.52851209e-01
 4.36912179e-01 4.20924471e-01 4.00866596e-01 3.60731725e-01
 3.49142995e-01 3.09289404e-01 2.72559629e-01 2.14120754e-01
 1.55073181e-01 1.48334695e-01 1.46445795e-01 1.36981125e-01
 1.28919895e-01 8.43046043e-02 2.60503847e-02 4.44089210e-16
 2.03431814e-16 1.44927594e-16 1.10143027e-16 7.66432169e-17
 6.40792104e-17 1.88283317e-17]
[0.00391921]
3.1210127770717755e-17 [1.00000000e+00 9.04018126e-01 6.57686519e-01 6.21612189e-01
 5.81157208e-01 5.07051467e-01 4.93428711e-01 4.81747024e-01
 4.69133775e-01 4.63951825e-01 4.58246888e-01 4.50058356e-01
 4.47705695e-01 4.34303971e-01 4.18157078e-01 3.95359751e-01
 3.61253165e-01 3.14672150e-01 2.75103706e-01 2.23689291e-01
 2.13070051e-01 1.60870042e-01 1.51797468e-01 1.47898759e-01
 1.21290712e-01 9.02837265e-02 2.28236720e-02 4.4408921

In [8]:
QPCAs=[qpca30,qpca40,qpca50,qpca60,qpca70]

In [27]:
dictionary_major = {}
dictionary_minor  = {}

ret_variance = [0.30,0.40,0.50,0.60,0.70]

for e,pca in enumerate(QPCAs):
    
    r = len(pca.estimate_fs[pca.estimate_fs < 0.20])
    
    p = len(pca.estimate_fs)

    out_threshold_list_major = []
    out_threshold_list_minor = []
    emp_distribution_major = []
    emp_distribution_minor = []
    
    for j in range(len(trains[e].drop(columns='41'))):
        sample = np.array(trains[e].drop(columns='41').iloc[j])
        y = np.dot(sample,pca.estimate_right_sv.T)
        s_major = np.sum(y[:np.searchsorted(stable_cumsum(pca.estimate_fs_ratio), ret_variance[e], side='right') + 1]**2 /
                         pca.estimate_fs[:np.searchsorted(stable_cumsum(pca.estimate_fs_ratio), ret_variance[e], side='right') + 1])
        
        if any(np.isclose(pca.estimate_fs,0)):
            s_minor = np.sum(y[p-r:np.where(np.isclose(pca.estimate_fs,0))[0][0]]**2 / 
                             pca.estimate_fs[p-r:np.where(np.isclose(pca.estimate_fs,0))[0][0]])
            
        else:
            s_minor = np.sum(y[p-r:]**2 / 
                             pca.estimate_fs[p-r:]) 
        

        emp_distribution_major.append(s_major)
        emp_distribution_minor.append(s_minor)

    for q in quantils:
        n_major = len(emp_distribution_major)
        n_minor = len(emp_distribution_minor)

        sort_major = sorted(emp_distribution_major)
        sort_minor = sorted(emp_distribution_minor)
        out_threshold_major = sort_major[int(n_major * q)]
        out_threshold_minor = sort_minor[int(n_minor * q)]
        out_threshold_list_major.append(out_threshold_major)
        out_threshold_list_minor.append(out_threshold_minor)

    dictionary_major.update({pca:out_threshold_list_major})
    dictionary_minor.update({pca:out_threshold_list_minor})


In [29]:
dictionary_minor

{qPCA(svd_solver='full'): [36.06112700390814,
  14.116244649104656,
  5.028756018323972,
  3.186696012634099,
  2.295928571883964,
  1.7878754397801546],
 qPCA(svd_solver='full'): [9.626239652324376,
  4.669760891020782,
  2.1629634945027374,
  1.3755369625598655,
  1.1052029137991255,
  0.8419601792525407],
 qPCA(svd_solver='full'): [37.6506405256557,
  7.967173320965593,
  6.326034938962397,
  6.189082420688406,
  6.104305085909954,
  6.0805470950574145],
 qPCA(svd_solver='full'): [10.629778055243452,
  8.014064752974459,
  3.8395599946061587,
  2.383050884757921,
  1.864004631770718,
  1.4973437040512643],
 qPCA(svd_solver='full'): [8.474928200915073,
  5.864637911879136,
  3.7258498839228675,
  2.301938619511212,
  1.4087695576980468,
  1.1653057209033613]}

In [32]:
classification_res = {}
for e,pca in enumerate(QPCAs):
    r = len(pca.estimate_fs[pca.estimate_fs < 0.20])
    p = len(pca.estimate_fs)

    acc_list  = []
    for threshold_major,threshold_minor in zip(dictionary_major[pca],dictionary_minor[pca]):
        print(threshold_major,threshold_minor)
        TP =[]
        FP = []
        TN = []
        FN = []

        for j in range(len(tests[e])):
            sample = np.array(tests[e].iloc[j])

            y = np.dot(sample, pca.estimate_right_sv.T)

            sum_major = np.sum(y[:np.searchsorted(stable_cumsum(pca.estimate_fs_ratio), ret_variance[e], side='right') + 1]**2 /
                             pca.estimate_fs[:np.searchsorted(stable_cumsum(pca.estimate_fs_ratio), ret_variance[e], side='right') + 1])

            if any(np.isclose(pca.estimate_fs,0)):
                sum_minor = np.sum(y[p-r:np.where(np.isclose(pca.estimate_fs,0))[0][0]]**2 / 
                                 pca.estimate_fs[p-r:np.where(np.isclose(pca.estimate_fs,0))[0][0]])

            else:
                sum_minor = np.sum(y[p-r:]**2 / 
                                 pca.estimate_fs[p-r:]) 


            if sum_major > threshold_major or sum_minor > threshold_minor:
                if test[e].iloc[j][41] == 'attack':
                    TP.append(j)
                else:
                    FP.append(j)
            else:
                if sum_major <= threshold_major and sum_minor <= threshold_minor:

                    if test[e].iloc[j][41] == 'attack':
                        FN.append(j)
                    else:
                        TN.append(j)

        accuracy = (len(TP))/(len(TP)+len(FN))
        print(accuracy)
        acc_list.append(accuracy)

    classification_res.update({pca:acc_list})

0.8373171230661575 36.06112700390814
0.8081211600212383
0.7345049376308077 14.116244649104656
0.9087507269095598
0.6555561005128155 5.028756018323972
0.9841723344542489
0.6110438220164558 3.186696012634099
0.986675431721069
0.5714950260393192 2.295928571883964
0.9882430279891785
0.5371731355013349 1.7878754397801546
0.9896842052034083
2.636352841186941 9.626239652324376
0.9842750600429782
1.2092807105872148 4.669760891020782
0.9859183415497409
0.8750576788611906 2.1629634945027374
0.9874352167867526
0.7259458942367213 1.3755369625598655
0.9891037795474655
0.6678619866361124 1.1052029137991255
0.9903931234989255
0.6145934252664504 0.8419601792525407
0.9912779673871824
4.4854200145851095 37.6506405256557
0.9843272074622715
2.1417274441506677 7.967173320965593
0.9868550772264213
1.841180720596575 6.326034938962397
0.9870320281099118
1.128916736040931 6.189082420688406
0.9874364872721757
0.8387328672458674 6.104305085909954
0.9884476351778356
0.6915191614458102 6.0805470950574145
0.9889784

In [33]:
components = [i for i in classification_res]
headers = ['FAR','PCC']
one_perc = [classification_res[i][0] for i in components]
two_perc = [classification_res[i][1] for i in components]
four_perc = [classification_res[i][2] for i in components]
six_perc = [classification_res[i][3] for i in components]
eight_perc = [classification_res[i][4] for i in components]
ten_perc = [classification_res[i][5] for i in components]

one_perc_avg = np.mean(one_perc)
one_perc_std = np.std(one_perc)

two_perc_avg = np.mean(two_perc)
two_perc_std = np.std(two_perc)

four_perc_avg = np.mean(four_perc)
four_perc_std = np.std(four_perc)

six_perc_avg = np.mean(six_perc)
six_perc_std = np.std(six_perc)


eight_perc_avg = np.mean(eight_perc)
eight_perc_std = np.std(eight_perc)

ten_perc_avg = np.mean(ten_perc)
ten_perc_std = np.std(ten_perc)

one_str = [str(one_perc_avg)+' +/- '+str(one_perc_std)]
two_str = [str(two_perc_avg)+' +/- '+str(two_perc_std)]
four_str = [str(four_perc_avg)+' +/- '+str(four_perc_std)]
six_str = [str(six_perc_avg)+' +/- '+str(six_perc_std)]
eight_str =[ str(eight_perc_avg)+' +/- '+str(eight_perc_std)]
ten_str = [str(ten_perc_avg)+' +/- '+str(ten_perc_std)]

one_str.insert(0,'1%')
two_str.insert(0,'2%')
four_str.insert(0,'4%')
six_str.insert(0,'6%')
eight_str.insert(0,'8%')
ten_str.insert(0,'10%')

print(tabulate([one_str, two_str, four_str, six_str, eight_str, ten_str], headers=headers))


FAR    PCC
-----  --------------------------------------------
1%     0.9496334443001564 +/- 0.07075900618628127
2%     0.9708768945695828 +/- 0.03106472074914475
4%     0.9866540849149004 +/- 0.0012520035749830057
6%     0.9880044085917026 +/- 0.0009047292509386547
8%     0.9890713928952073 +/- 0.0007703466090644553
10%    0.9913420383440157 +/- 0.0024135119895022515
